# Covid19 Exploratory Data Analysis

#### Importing the required libraries 

In [3]:
import pandas as pd
from pandasql import sqldf

#### Importing files. Since the files are big, "low_memory = False" is used.

In [7]:
covid_deaths = pd.read_csv("CovidDeaths.csv", low_memory = False)
covid_vaccinations = pd.read_csv("CovidVaccinations.csv", low_memory = False)

#### Initialize the pandasql environment

In [12]:
pysqldf = sqldf('SELECT COUNT(*) FROM covid_deaths;', globals())
result1 = pysqldf
print("Query 1 Result:")
print(result1)

Query 1 Result:
   COUNT(*)
0    362259


In [15]:
# Query 2
query2 = '''
    SELECT * FROM covid_deaths
    WHERE continent IS NOT NULL 
    ORDER BY 3, 4;
'''

result2 = sqldf(query2, globals())
print("\nQuery 2 Result:")
print(result2)


Query 2 Result:
       iso_code continent     location      date  population  total_cases  \
0           AFG      Asia  Afghanistan  01-01-21    41128772      52513.0   
1           AFG      Asia  Afghanistan  01-01-22    41128772     158059.0   
2           AFG      Asia  Afghanistan  01-01-23    41128772     207579.0   
3           AFG      Asia  Afghanistan  01-02-20    41128772          NaN   
4           AFG      Asia  Afghanistan  01-02-21    41128772      55059.0   
...         ...       ...          ...       ...         ...          ...   
344990      ZWE    Africa     Zimbabwe  31-10-22    16320539     258169.0   
344991      ZWE    Africa     Zimbabwe  31-10-23    16320539     265880.0   
344992      ZWE    Africa     Zimbabwe  31-12-20    16320539      13625.0   
344993      ZWE    Africa     Zimbabwe  31-12-21    16320539     211728.0   
344994      ZWE    Africa     Zimbabwe  31-12-22    16320539     260100.0   

        new_cases  new_cases_smoothed  total_deaths  new_d

In [19]:
# Query 3
query3 = '''
    WITH PopvsVac AS (
        SELECT
            dea.continent,
            dea.location,
            dea.date,
            dea.population,
            vac.new_vaccinations,
            SUM(CAST(vac.new_vaccinations AS INTEGER)) OVER (PARTITION BY dea.location ORDER BY dea.location, dea.date) as RollingPeopleVaccinated
        FROM
            covid_deaths dea
            JOIN covid_vaccinations vac ON dea.location = vac.location AND dea.date = vac.date
        WHERE
            dea.continent IS NOT NULL
    )
    SELECT
        *,
        (RollingPeopleVaccinated / Population) * 100 as VaccinationPercentage
    FROM
        PopvsVac;
'''

result3 = sqldf(query3, globals())
print("\nQuery 3 Result:")
print(result3)



Query 3 Result:
       continent     location      date  population  new_vaccinations  \
0           Asia  Afghanistan  01-01-21    41128772               NaN   
1           Asia  Afghanistan  01-01-22    41128772               NaN   
2           Asia  Afghanistan  01-01-23    41128772               NaN   
3           Asia  Afghanistan  01-02-20    41128772               NaN   
4           Asia  Afghanistan  01-02-21    41128772               NaN   
...          ...          ...       ...         ...               ...   
344990    Africa     Zimbabwe  31-10-22    16320539               NaN   
344991    Africa     Zimbabwe  31-10-23    16320539               NaN   
344992    Africa     Zimbabwe  31-12-20    16320539               NaN   
344993    Africa     Zimbabwe  31-12-21    16320539               NaN   
344994    Africa     Zimbabwe  31-12-22    16320539               NaN   

        RollingPeopleVaccinated  VaccinationPercentage  
0                           NaN                  

In [21]:
# Query 4
query4 = '''
    SELECT
        location,
        SUM(new_deaths) as TotalDeathCount
    FROM
        covid_deaths
    WHERE
        continent IS NULL
        AND location NOT IN ('World', 'European Union', 'International')
    GROUP BY
        location
    ORDER BY
        TotalDeathCount DESC;
'''

result4 = sqldf(query4, globals())
print("\nQuery 4 Result:")
print(result4)
result4.to_csv("Tableau Table 2.csv", index=False)


Query 4 Result:
              location  TotalDeathCount
0          High income        2917813.0
1  Upper middle income        2664957.0
2               Europe        2085294.0
3                 Asia        1635885.0
4        North America        1606914.0
5        South America        1357606.0
6  Lower middle income        1340820.0
7               Africa         259047.0
8           Low income          48014.0
9              Oceania          30814.0
